
## Overview

Este notebook contém o tratamento da tabela de Empresas Inidôneas e Suspensas, mantida pela CGU. O objetivo é verificar se há gastos no cartão corporativo efetuado com alguma empresa sancionada. Para limitar o escopo, só serão mantidas as Pessoas Jurídicas.

In [0]:
# File location and type
file_location = "/FileStore/tables/CEIS.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ";"
encode = 'latin1'


# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("encoding", encode) \
  .load(file_location)

# display(df)

In [0]:
df = df.filter(df["TIPO DE PESSOA"] == "J")

In [0]:
# retirar colunas desnecessárias
cols_to_drop = ["CADASTRO", "NOME INFORMADO PELO ÓRGÃO SANCIONADOR", 
                "NOME FANTASIA - CADASTRO RECEITA", "DETALHAMENTO", 
                "DATA DO TRÂNSITO EM JULGADO", "ABRAGÊNCIA DEFINIDA EM DECISÃO JUDICIAL"]
df = df.drop(*cols_to_drop)

In [0]:
new_col_names = ["cadastro", "tipo_pessoa", "cnpj_sanc", 
                 "nome_sanc", "razao_social", "n_processo", 
                 "categ_sancao", "dt_in_san", "dt_fin_san", 
                 "dt_pub", "pub", "org_sancionador", "uf_orgao",
                 "esfera_org", "fund_legal"]

df = df.toDF(*new_col_names)

Dividir tabela: uma tabela conterá apenas os dados cadastrais das empresas, e a outra, os dados de sanção

In [0]:
df_empresas = df[["cnpj_sanc", "nome_sanc", "razao_social"]]

In [0]:
new_cols_empresas = ["cnpj", "nome", "razao_social"]
df_empresas = df_empresas.toDF(*new_cols_empresas).dropDuplicates(["cnpj"])

In [0]:
cols_to_drop = ["nome_sanc", "razao_social"]
df = df.drop(*cols_to_drop)

In [0]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "ceis"
# df.write.format("parquet").saveAsTable(permanent_table_name)
df.write.mode("overwrite").saveAsTable(permanent_table_name)

In [0]:
%sql
select * from ceis limit 5

cadastro,tipo_pessoa,cnpj_sanc,nome_sanc,razao_social,n_processo,categ_sancao,dt_in_san,dt_fin_san,dt_pub,pub,org_sancionador,uf_orgao,esfera_org,fund_legal
106167,J,00368364000129,null,null,00032225520098260197,Impedimento/proibição de contratar com prazo determinado,27/08/2019,27/08/2024,null,Sem Informação,Justiça Federal de Primeiro Grau em São Paulo,SP,FEDERAL,"LEI 8429 - ART. 12 - INDEPENDENTEMENTE DAS SANÇÕES PENAIS, CIVIS E ADMINISTRATIVAS PREVISTAS NA LEGISLAÇÃO ESPECÍFICA, ESTÁ O RESPONSÁVEL PELO ATO DE IMPROBIDADE SUJEITO ÀS SEGUINTES COMINAÇÕES, QUE PODEM SER APLICADAS ISOLADA OU CUMULATIVAMENTE, DE ACORDO COM A GRAVIDADE DO FATO: I - NA HIPÓTESE DO ART. 9°, PERDA DOS BENS OU VALORES ACRESCIDOS ILICITAMENTE AO PATRIMÔNIO, RESSARCIMENTO INTEGRAL DO DANO, QUANDO HOUVER, PERDA DA FUNÇÃO PÚBLICA, SUSPENSÃO DOS DIREITOS POLÍTICOS DE OITO A DEZ ANOS, PAGAMENTO DE MULTA CIVIL DE ATÉ TRÊS VEZES O VALOR DO ACRÉSCIMO PATRIMONIAL E PROIBIÇÃO DE CONTRATAR COM O PODER PÚBLICO OU RECEBER BENEFÍCIOS OU INCENTIVOS FISCAIS OU CREDITÍCIOS, DIRETA OU INDIRETAMENTE, AINDA QUE POR INTERMÉDIO DE PESSOA JURÍDICA DA QUAL SEJA SÓCIO MAJORITÁRIO, PELO PRAZO DE DEZ ANOS; II - NA HIPÓTESE DO ART. 10, RESSARCIMENTO INTEGRAL DO DANO, PERDA DOS BENS OU VALORES ACRESCIDOS ILICITAMENTE AO PATRIMÔNIO, SE CONCORRER ESTA CIRCUNSTÂNCIA, PERDA DA FUNÇÃO PÚBLICA, SUSPENSÃO DOS DIREITOS POLÍTICOS DE CINCO A OITO ANOS, PAGAMENTO DE MULTA CIVIL DE ATÉ DUAS VEZES O VALOR DO DANO E PROIBIÇÃO DE CONTRATAR COM O PODER PÚBLICO OU RECEBER BENEFÍCIOS OU INCENTIVOS FISCAIS OU CREDITÍCIOS, DIRETA OU INDIRETAMENTE, AINDA QUE POR INTERMÉDIO DE PESSOA JURÍDICA DA QUAL SEJA SÓCIO MAJORITÁRIO, PELO PRAZO DE CINCO ANOS; III - NA HIPÓTESE DO ART. 11, RESSARCIMENTO INTEGRAL DO DANO, SE HOUVER, PERDA DA FUNÇÃO PÚBLICA, SUSPENSÃO DOS DIREITOS POLÍTICOS DE TRÊS A CINCO ANOS, PAGAMENTO DE MULTA CIVIL DE ATÉ CEM VEZES O VALOR DA REMUNERAÇÃO PERCEBIDA PELO AGENTE E PROIBIÇÃO DE CONTRATAR COM O PODER PÚBLICO OU RECEBER BENEFÍCIOS OU INCENTIVOS FISCAIS OU CREDITÍCIOS, DIRETA OU INDIRETAMENTE, AINDA QUE POR INTERMÉDIO DE PESSOA JURÍDICA DA QUAL SEJA SÓCIO MAJORITÁRIO, PELO PRAZO DE TRÊS ANOS."
303288,J,32769825000161,null,null,23314.000919.2022,Impedimento/proibição de contratar com prazo determinado,24/10/2022,24/10/2024,null,Sem Informação,"INST.FED.DE EDUC.,CIENC.E TEC. DE SÃO PAULO",SP,FEDERAL,"LEI 10520 - ART. 7º - QUEM, CONVOCADO DENTRO DO PRAZO DE VALIDADE DA SUA PROPOSTA, NÃO CELEBRAR O CONTRATO, DEIXAR DE ENTREGAR OU APRESENTAR DOCUMENTAÇÃO FALSA EXIGIDA PARA O CERTAME, ENSEJAR O RETARDAMENTO DA EXECUÇÃO DE SEU OBJETO, NÃO MANTIVER A PROPOSTA, FALHAR OU FRAUDAR NA EXECUÇÃO DO CONTRATO, COMPORTAR-SE DE MODO INIDÔNEO OU COMETER FRAUDE FISCAL, FICARÁ IMPEDIDO DE LICITAR E CONTRATAR COM A UNIÃO, ESTADOS, DISTRITO FEDERAL OU MUNICÍPIOS E, SERÁ DESCREDENCIADO NO SICAF, OU NOS SISTEMAS DE CADASTRAMENTO DE FORNECEDORES A QUE SE REFERE O INCISO XIV DO ART. 4O DESTA LEI, PELO PRAZO DE ATÉ 5 (CINCO) ANOS, SEM PREJUÍZO DAS MULTAS PREVISTAS EM EDITAL E NO CONTRATO E DAS DEMAIS COMINAÇÕES LEGAIS."
92682,J,04267536000129,null,null,0200170175165,Declaração de Inidoneidade com prazo determinado,16/03/2018,null,16/03/2018,Diário Oficial do Estado Seção LICITACOES Pagina 11,Governo do Estado da Bahia (BA),BA,ESTADUAL,"LEI 9433 (BA) - ART. 186, III - AO CANDIDATO A CADASTRAMENTO, AO LICITANTE E AO CONTRATADO, QUE INCORRAM NAS FALTAS PREVISTAS NESTA LEI, APLICAM-SE, SEGUNDO A NATUREZA E A GRAVIDADE DA FALTA, ASSEGURADA A DEFESA PRÉVIA, AS SEGUINTES SANÇÕES: III- DECLARAÇÃO DE INIDONEIDADE PARA LICITAR OU CONTRATAR COM A ADMINISTRAÇÃO PÚBLICA, ENQUANTO PERDURAREM OS MOTIVOS DETERMINANTES DESTA PUNIÇÃO E ATÉ QUE SEJA PROMOVIDA SUA REABILITAÇÃO PERANTE A ADMINISTRAÇÃO PÚBLICA ESTADUAL"
300398,J,27178359000100,null,null,579/2022,Suspensão,18/11/2022,17/11/2024,null,Sem Informação,ESTADO DO RIO DE JANEIRO,RJ,ESTADUAL,"LEI 8666 - ART. 87, III - PELA INEXECUÇÃO TOTAL OU PARCIAL DO CONTRATO A ADMINIST

In [0]:
%sql
DROP TABLE IF EXISTS cadastro_empresas

In [0]:
permanent_table_name = "cadastro_empresas"
# df_empresas.write.format("parquet").saveAsTable(permanent_table_name)
df_empresas.write.mode("overwrite").saveAsTable(permanent_table_name)

In [0]:
%sql
select * from cadastro_empresas limit 5

cnpj,nome,razao_social
00000104000104,INSTITUTO GRANDE VITÓRIA,INSTITUTO GRANDE VITORIA
00000113000197,ASSOCIAÇÃO GRUPO DE MAES SOLIDARIAS BEM VIVER,ASSOCIACAO GRUPO DE MAES SOLIDARIAS BEM VIVER
00000145000192,ASSOCIAÇÃO CLUBE DE MÃES RITA DE CÁSSIA,ASSOCIACAO E CLUBE DE MAES 'RITA DE CASSIA'
00002776000140,DEUTSCHEBRAS COMERCIAL E ENGENHARIA LTDA,DEUTSCHEBRAS COMERCIAL E ENGENHARIA LTDA
00004564000100,IGREJA EVANGÉLICA ASSEMBLÉIA DE DEUS,IGREJA EVANGELICA ASSEMBLEIA DE DEUS
